<a href="https://colab.research.google.com/github/dkjpzore/Boston-Housing-/blob/main/IMDB_%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C_%EC%BD%98%ED%85%90%EC%B8%A0_%EA%B8%B0%EB%B0%98_%ED%95%84%ED%84%B0%EB%A7%81_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url1 = ('https://raw.githubusercontent.com/sd12832/MoViZ/master/' +
        'src/tmdb_5000_credits.csv')
df1 = pd.read_csv(url1)
print(df1.shape)
df1.head()

(4803, 4)


movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

In [ ]:
url2 = ('https://raw.githubusercontent.com/sd12832/MoViZ/master/' +
        'src/tmdb_5000_movies.csv')
df2 = pd.read_csv(url2)
print(df2.shape)
df2.head(2)

(4803, 20)


budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   

                                       homepage     id  \
0                   http://www.avatarmovie.com/  19995   
1  http://disney.go.com/disneypictures/pirates/    285   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   

                                      title  vote_average  vote_count  
0                                    Avatar           7.2       11800  
1  Pirates of the Caribbean: At World's End           6.9        4500

In [ ]:
#credits와 movies 병합 (id 기준)
df1.rename(columns={'movie_id': 'id'}, inplace=True)
df = df2.merge(df1, on='id')

In [ ]:
# 필요한 컬럼만 추출
df = df[['title_x', 'vote_average', 'vote_count', 'original_language']]

In [ ]:
#평점 가중치 계산 (IMDB 방식: weighted rating)
# 전체 평균 평점
C = df['vote_average'].mean()
# 상위 영화에 포함될 최소 투표 수 (예: 상위 90% 기준)
m = df['vote_count'].quantile(0.9)

In [ ]:
# 필터링: m 이상 투표 받은 영화만
qualified = df[df['vote_count'] >= m].copy()

In [ ]:
# 가중 평균 공식 적용
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m)) * R + (m / (v + m)) * C

qualified['score'] = qualified.apply(weighted_rating, axis=1)

In [ ]:
# 영어권 / 비영어권 상위 10개 추출
top_en = qualified[qualified['original_language'] == 'en'].sort_values(by='score', ascending=False).head(10)
top_non_en = qualified[qualified['original_language'] != 'en'].sort_values(by='score', ascending=False).head(10)


In [ ]:
# 결과 출력
print("영어권 영화 Top 10:")
print(top_en[['title_x', 'score', 'vote_average', 'vote_count']])

print("비영어권 영화 Top 10:")
print(top_non_en[['title_x', 'score', 'original_language', 'vote_average', 'vote_count']])

영어권 영화 Top 10:
                                            title_x     score  vote_average  \
1881                       The Shawshank Redemption  8.059258           8.5   
662                                      Fight Club  7.939256           8.3   
65                                  The Dark Knight  7.920020           8.2   
3232                                   Pulp Fiction  7.904645           8.3   
96                                        Inception  7.863239           8.1   
3337                                  The Godfather  7.851236           8.4   
95                                     Interstellar  7.809479           8.1   
809                                    Forrest Gump  7.803188           8.2   
329   The Lord of the Rings: The Return of the King  7.727243           8.1   
1990                        The Empire Strikes Back  7.697884           8.2   

      vote_count  
1881        8205  
662         9413  
65         12002  
3232        8428  
96         13752  
3

In [ ]:
# 1. 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 2. 데이터 불러오기
url = 'https://raw.githubusercontent.com/sd12832/MoViZ/master/src/tmdb_5000_movies.csv'
df = pd.read_csv(url)

# 3. 결측값 제거
df['overview'] = df['overview'].fillna('')

In [ ]:
# 4. TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words='english')  # 영어 불용어 제거
tfidf_matrix = tfidf.fit_transform(df['overview'])  # 줄거리 벡터화

In [ ]:
# 5. 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# 6. 영화 제목 → 인덱스 매핑
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [ ]:
# 7. 추천 함수 정의
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]  # 입력 영화의 인덱스
    sim_scores = list(enumerate(cosine_sim[idx]))  # 모든 영화와의 유사도
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # 유사도 높은 순 정렬
    sim_scores = sim_scores[1:11]  # 자기 자신 제외하고 상위 10개
    movie_indices = [i[0] for i in sim_scores]  # 추천 영화 인덱스
    return df[['title', 'overview']].iloc[movie_indices]

In [ ]:
# 8. 추천 테스트
recommended = get_recommendations('The Dark Knight Rises')  # 추천 기준 영화
print("'The Dark Knight Rises'와 줄거리가 비슷한 영화 Top 10:")
print(recommended)

'The Dark Knight Rises'와 줄거리가 비슷한 영화 Top 10:
                                        title  \
65                            The Dark Knight   
299                            Batman Forever   
428                            Batman Returns   
1359                                   Batman   
3854  Batman: The Dark Knight Returns, Part 2   
119                             Batman Begins   
2507                                Slow Burn   
9          Batman v Superman: Dawn of Justice   
1181                                      JFK   
210                            Batman & Robin   

                                               overview  
65    Batman raises the stakes in his war on crime. ...  
299   The Dark Knight of Gotham City confronts a das...  
428   Having defeated the Joker, Batman now faces th...  
1359  The Dark Knight of Gotham City begins his war ...  
3854  Batman has stopped the reign of terror that Th...  
119   Driven by tragedy, billionaire Bruce Wayne ded...  
2507  A d

In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 2. 데이터 불러오기 및 병합
credits = pd.read_csv('https://raw.githubusercontent.com/sd12832/MoViZ/master/src/tmdb_5000_credits.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/sd12832/MoViZ/master/src/tmdb_5000_movies.csv')

credits.columns = ['id' if col == 'movie_id' else col for col in credits.columns]
df = movies.merge(credits, on='id')

In [ ]:
# 3. 주요 컬럼 선택
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(ast.literal_eval)

In [ ]:
# 4. 주요 정보 추출 함수 정의
def get_director(crew):
    for member in crew:
        if member['job'] == 'Director':
            return member['name']
    return np.nan

def get_list(obj):
    return [d['name'] for d in obj][:3]  # 상위 3개만 사용 (과적합 방지)

df['director'] = df['crew'].apply(get_director)
for feature in ['cast', 'keywords', 'genres']:
    df[feature] = df[feature].apply(get_list)

In [ ]:
# 5. 소문자 변환 및 공백 제거
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        return str.lower(x.replace(" ", ""))
    else:
        return ''

df['cast'] = df['cast'].apply(clean_data)
df['keywords'] = df['keywords'].apply(clean_data)
df['genres'] = df['genres'].apply(clean_data)
df['director'] = df['director'].apply(clean_data)

In [ ]:
# 6. soup 만들기
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + \
           ' '.join(x['genres']) + ' ' + x['director']

df['soup'] = df.apply(create_soup, axis=1)


In [ ]:
# 7. CountVectorizer로 벡터화
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [ ]:
# 8. 코사인 유사도 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# 9. 영화 제목 → 인덱스 매핑
df = df.reset_index()
indices = pd.Series(df.index, index=df['title_x'])

In [ ]:
# 10. 추천 함수 정의
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # 자기 자신 제외
    movie_indices = [i[0] for i in sim_scores]
    return df[['title_x', 'vote_average', 'vote_count']].iloc[movie_indices]

In [ ]:
# 11. 예시 실행
print("The Dark Knight Rises'와 메타데이터 기반 유사 영화 Top 10:")
print(get_recommendations('The Dark Knight Rises'))

The Dark Knight Rises'와 메타데이터 기반 유사 영화 Top 10:
                       title_x  vote_average  vote_count
65             The Dark Knight           8.2       12002
119              Batman Begins           7.5        7359
4638  Amidst the Devil's Wings           0.0           0
1196              The Prestige           8.0        4391
3073         Romeo Is Bleeding           5.7          36
3326            Black November           6.4           4
1503                    Takers           6.0         394
1986                    Faster           6.1         568
303                   Catwoman           4.2         808
747             Gangster Squad           6.2        1778
